In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.backends import cudnn

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
from typing import List, Any

# Set seed

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
cudnn.deterministic = True

# Get tokenizer

In [5]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     |████████████████████████████████| 13.6 MB 877 kB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     |████████████████████████████████| 18.8 MB 12.0 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


# Tokenization function

In [7]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

def tokenize_de(text: str) -> List[str]:
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    :param text: German string
    :return: list of tokens
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text: str) -> List[str]:
    """
    Tokenizes English text from a string into a list of strings (tokens) and reverses it
    :param text: English string
    :return: list of tokens
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# Make object that will contain the tokenized text(including special tokens)

In [8]:
SRC = Field(tokenize = tokenize_de,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

TRG = Field(tokenize=tokenize_en,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

# Train, validation, test set split

In [9]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields= (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:04<00:00, 299kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 78.5kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 116kB/s] 


In [10]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


# Build vocab
Using the `min_freq` argument, we only allow tokens that appear at least 2 times to appear in our vocabulary. Tokens that appear once are converted to `<unk>`.
It is important to note that our vocabulary should only be built from the training set and not validation/test set. This prevents *information leakage* into our model, giving us artificially inflated validation/test scores.

In [11]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [12]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7853
Unique tokens in target (en) vocabulary: 5893


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Batching
We define the batch size and an iterator that will split each dataset into batches. This iterator should also be able to move the batches the designated device.

In [14]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)

# Seq2Seq Model
## Encoder

## Decoder

## Encapsulation